In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.utils import resample
from collections import Counter

import nltk
from apyori import apriori
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import wordnet 
from nltk.corpus import sentiwordnet as swn
from collections import Counter
from textblob import TextBlob

In [48]:
with open('NRC-Emotion-Lexicon-Wordlevel-v0.92.csv', 'r') as f:
    df = pd.read_csv(f, delimiter='\t', header=0, names=['word', 'emotion', 'relation'])

# Create lists of emotion words
#(anger, fear, anticipation, trust, surprise, sadness, joy, and disgust)
anticipation_NRC = list(df[(df['emotion']=='anticipation') & (df['relation']!=0)]['word'])
anger_NRC = list(df[(df['emotion']=='anger') & (df['relation']!=0)]['word'])
fear_NRC = list(df[(df['emotion']=='fear') & (df['relation']!=0)]['word'])
trust_NRC = list(df[(df['emotion']=='trust') & (df['relation']!=0)]['word'])
surprise_NRC = list(df[(df['emotion']=='surprise') & (df['relation']!=0)]['word'])
sadness_NRC = list(df[(df['emotion']=='sadness') & (df['relation']!=0)]['word'])
joy_NRC = list(df[(df['emotion']=='joy') & (df['relation']!=0)]['word'])
disgust_NRC = list(df[(df['emotion']=='disgust') & (df['relation']!=0)]['word'])
neg_NRC = list(df[(df['emotion']=='negative') & (df['relation']!=0)]['word'])
pos_NRC = list(df[(df['emotion']=='positive') & (df['relation']!=0)]['word'])

In [107]:
sad = pd.read_csv('sad.csv', header=None)
sad_LIWC = list(sad[0])
anger = pd.read_csv('anger.csv', header=None)
anger_LIWC = (list(anger_LIWC[0])+list(anger_LIWC[1]))
anxiety = pd.read_csv('anxiety.csv', header=None)
anxiety_LIWC = list(anxiety[0])
negemo = pd.read_csv('negemo.csv', header=None)
negemo_LIWC =(list(negemo[0])+list(negemo[1])+ list(negemo[2])+list(negemo[3])+list(negemo[4]))
posemo = pd.read_csv('posemo.csv', header=None)
posemo_LIWC =(list(posemo[0])+list(posemo[1])+ list(posemo[2])+list(posemo[3])+list(posemo[4]))
negat = pd.read_csv('negat.csv', header=None)
negat_LIWC = list(negat[0])
adj = pd.read_csv('adj.csv', header=None)
adj_LIWC =(list(adj[0])+list(adj[1])+ list(adj[2])+list(adj[3])+list(adj[4]))
cogproc = pd.read_csv('LIWC - cogproc.csv', header=None)
cogproc_LIWC = (list(cogproc[0])+list(cogproc[1])+ list(cogproc[2])+list(cogproc[3])+list(cogproc[4])+list(cogproc[5]))
percep = pd.read_csv('LIWC - percept.csv', header=None)
percep_LIWC = (list(percep[0])+list(percep[1])+ list(percep[2]))
space = pd.read_csv('LIWC - space.csv', header=None)
space_LIWC = (list(space[0])+list(space[1])+ list(space[2]))
time = pd.read_csv('LIWC - time.csv', header=None)
time_LIWC = list(time[0])+list(time[1])
cause = pd.read_csv('LIWC - cause.csv', header=None)
cause_LIWC=list(cause[0])
comp = pd.read_csv('LIWC - comparatives (1).csv', header=None)
comp_LIWC=(list(comp[0])+list(comp[1])+ list(comp[2]))
money = pd.read_csv('LIWC - money (1).csv', header=None)
money_LIWC=(list(money[0])+list(money[1]))
BOW1 = pd.read_csv('wordcounts_bow_d300_h1024_64_epoch10_1000.csv')
BOW2 = pd.read_csv('wordcounts_bow_d300_h1024_64_epoch10_1910.csv')

In [60]:
fp= ['we','us','our']
tp=['he','she','it','him','her','his','hers','its','they','them','their','theirs']            
narr= ['say','tell','said','told','saying','telling', 'then', 'after', 'before','initially', 'first',\
       'next', 'while', 'during', 'finally', 'eventually','end', 'start']

In [40]:
BOW.head()

,Unnamed: 0,pred,target,vocab,attr
0,0,5,5,best,0.699070
1,1,5,5,great,0.590527
2,2,5,5,perfect,1.000000
3,3,5,5,perfect,1.000000
4,4,5,5,disappointment,-0.889639


In [20]:
BOW = pd.concat([BOW1,BOW2])

In [34]:
corrects_2 = BOW[(BOW['pred']==2) & (BOW['target']==2)]
corrects_3 = BOW[(BOW['pred']==3) & (BOW['target']==3)]
corrects_4 = BOW[(BOW['pred']==4) & (BOW['target']==4)]
corrects_5 = BOW[(BOW['pred']==5) & (BOW['target']==5)]

incorrects_2 = BOW[(BOW['pred']==2) & (BOW['target']!=2)]
incorrects_3 = BOW[(BOW['pred']==3) & (BOW['target']!=3)]
incorrects_4 = BOW[(BOW['pred']==4) & (BOW['target']!=4)]
incorrects_5 = BOW[(BOW['pred']==5) & (BOW['target']!=5)]

In [35]:
print('Number of correct 2: ' + str(len(corrects_2)))
print('Number of correct 3: ' + str(len(corrects_3)))
print('Number of correct 4: ' + str(len(corrects_4)))
print('Number of correct 5: ' + str(len(corrects_5)))

print('Number of incorrect 2: ' + str(len(incorrects_2)))
print('Number of incorrect 3: ' + str(len(incorrects_3)))
print('Number of incorrect 4: ' + str(len(incorrects_4)))
print('Number of incorrect 5: ' + str(len(incorrects_5)))

Number of correct 2: 995
Number of correct 3: 524
Number of correct 4: 1677
Number of correct 5: 4621
Number of incorrect 2: 462
Number of incorrect 3: 391
Number of incorrect 4: 1241
Number of incorrect 5: 1325


In [ ]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [108]:
print("Rating 2 vs LIWC")
print("Overlap between Sad and + 2's: " 
      + str(intersection(sad_LIWC, list(corrects_2[corrects_2['attr']>0]['vocab']))))
print("Overlap between Sad and - 2's: " 
      + str(intersection(sad_LIWC, list(corrects_2[corrects_2['attr']<=0]['vocab']))))
print("Overlap between Anger and + 2's: " 
      + str(intersection(anger_LIWC, list(corrects_2[corrects_2['attr']>0]['vocab']))))
print("Overlap between Anger and - 2's: " 
      + str(intersection(anger_LIWC, list(corrects_2[corrects_2['attr']<=0]['vocab']))))
print("Overlap between Anxiety and + 2's: " 
      + str(intersection(anxiety_LIWC, list(corrects_2[corrects_2['attr']>0]['vocab']))))
print("Overlap between Anxiety and - 2's: " 
      + str(intersection(anxiety_LIWC, list(corrects_2[corrects_2['attr']<=0]['vocab']))))
print("Overlap between Negemo and + 2's: " 
      + str(intersection(negemo_LIWC, list(corrects_2[corrects_2['attr']>0]['vocab']))))
print("Overlap between Negemo and - 2's: " 
      + str(intersection(negemo_LIWC, list(corrects_2[corrects_2['attr']<=0]['vocab']))))
print("Overlap between Posemo and + 2's: " 
      + str(intersection(posemo_LIWC, list(corrects_2[corrects_2['attr']>0]['vocab']))))
print("Overlap between Posemo and - 2's: " 
      + str(intersection(posemo_LIWC, list(corrects_2[corrects_2['attr']<=0]['vocab']))))
print("Overlap between negat and + 2's: " 
      + str(intersection(negat_LIWC, list(corrects_2[corrects_2['attr']>0]['vocab']))))
print("Overlap between negat and - 2's: " 
      + str(intersection(negat_LIWC, list(corrects_2[corrects_2['attr']<=0]['vocab']))))
print("Overlap between adj and + 2's: " 
      + str(intersection(adj_LIWC, list(corrects_2[corrects_2['attr']>0]['vocab']))))
print("Overlap between adj and - 2's: " 
      + str(intersection(adj_LIWC, list(corrects_2[corrects_2['attr']<=0]['vocab']))))

Rating 2 vs LIWC
Overlap between Sad and + 2's: []
Overlap between Sad and - 2's: []
Overlap between Anger and + 2's: ['crappy', 'rude', 'rudely', 'protesting', 'sucks']
Overlap between Anger and - 2's: []
Overlap between Anxiety and + 2's: ['horrible']
Overlap between Anxiety and - 2's: []
Overlap between Negemo and + 2's: ['worse', 'gross', 'worst', 'crappy', 'lies', 'serious', 'pathetic', 'unsafe', 'unpleasant', 'unfriendly', 'poor', 'horrible', 'rude', 'rudely', 'protesting', 'ignored', 'sad', 'pitiful', 'mess', 'terrible', 'awful', 'sorry', 'sucks', 'poorly', 'bad']
Overlap between Negemo and - 2's: []
Overlap between Posemo and + 2's: ['support', 'favor', 'trust', 'excited', 'positive', 'favors', 'luck']
Overlap between Posemo and - 2's: ['perfect', 'gorgeous', 'comfortable', 'best', 'loved', 'fantastic', 'ok', 'great', 'nice', 'decent', 'amazing', 'lovely', 'fine', 'good', 'love', 'wonderful', 'excellent', 'beautiful', 'nicely']
Overlap between negat and + 2's: ['didnt', 'nowher

In [56]:
print("Rating 2 vs NRC")
print("Overlap between Sad and + 2's: " 
      + str(intersection(list(sadness_NRC), list(corrects_2[corrects_2['attr']>0]['vocab']))) + '\n')
print("Overlap between Sad and - 2's: " 
      + str(intersection(list(sadness_NRC), list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Anger and + 2's: " 
      + str(intersection(list(anger_NRC), list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Anger and - 2's: " 
      + str(intersection(list(anger_NRC), list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Fear and + 2's: " 
      + str(intersection(list(fear_NRC), list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Fear and - 2's: " 
      + str(intersection(list(fear_NRC), list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Trust and + 2's: " 
      + str(intersection(list(trust_NRC), list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Trust and - 2's: " 
      + str(intersection(list(trust_NRC), list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Surprise and + 2's: " 
      + str(intersection(list(surprise_NRC), list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Surprise and - 2's: " 
      + str(intersection(list(surprise_NRC), list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Anticipation and + 2's: " 
      + str(intersection(list(anticipation_NRC), list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Anticipation and - 2's: " 
      + str(intersection(list(anticipation_NRC), list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Disgust and + 2's: " 
      + str(intersection(list(disgust_NRC), list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Disgust and - 2's: " 
      + str(intersection(list(disgust_NRC), list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Neg and + 2's: " 
      + str(intersection(list(neg_NRC), list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Neg and - 2's: " 
      + str(intersection(list(neg_NRC), list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Pos and + 2's: " 
      + str(intersection(list(pos_NRC), list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Pos and - 2's: " 
      + str(intersection(list(pos_NRC), list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')

Rating 2 vs NRC
Overlap between Sad and + 2's: ['worse', 'disastrous', 'beg', 'depressing', 'overpriced', 'unbearable', 'prison', 'homeless', 'incompetent', 'dire', 'dishonest', 'dilapidated', 'destroyed', 'pathetic', 'unpleasant', 'miserably', 'unfriendly', 'broken', 'mistake', 'nasty', 'discourage', 'worn', 'ashamed', 'disappointment', 'uncaring', 'dark', 'ruined', 'disaster', 'dreadful', 'refused', 'unacceptable', 'terrible', 'disappointing', 'awful', 'inadequate', 'flop', 'unsatisfied', 'miserable', 'chaos', 'inexcusable', 'disappointed', 'bad']

Overlap between Sad and - 2's: ['negative', 'lovely']

Overlap between Anger and + 2's: ['disastrous', 'strike', 'overpriced', 'prison', 'homeless', 'incompetent', 'battle', 'blatant', 'stolen', 'dishonest', 'destroyed', 'unfriendly', 'broken', 'nasty', 'disgusting', 'horrible', 'disdain', 'uncaring', 'arrogant', 'unsympathetic', 'ruined', 'disaster', 'dreadful', 'terrible', 'monstrosity', 'ordeal', 'awful', 'surly', 'noisy', 'miserable', 

In [63]:
print("2's and Narrative")
print("Overlap between First Person and + 2's: " 
      + str(intersection(fp, list(corrects_2[corrects_2['attr']>0]['vocab']))) + '\n')
print("Overlap between First Person and - 2's: " 
      + str(intersection(fp, list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Third Person and + 2's: " 
      + str(intersection(tp, list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Third Person and - 2's: " 
      + str(intersection(tp, list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Narrative Seq and + 2's: " 
      + str(intersection(narr, list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Narrative Seq and - 2's: " 
      + str(intersection(narr, list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')

2's and Narrative
Overlap between First Person and + 2's: []

Overlap between First Person and - 2's: []

Overlap between Third Person and + 2's: []

Overlap between Third Person and - 2's: []

Overlap between Narrative Seq and + 2's: []

Overlap between Narrative Seq and - 2's: []



In [109]:
print("Overlap between Cognitive and + 2's: " 
      + str(intersection(cogproc_LIWC, list(corrects_2[corrects_2['attr']>0]['vocab']))) + '\n')
print("Overlap between Cognitive Person and - 2's: " 
      + str(intersection(cogproc_LIWC, list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Perception Person and + 2's: " 
      + str(intersection(percep_LIWC, list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Perception and - 2's: " 
      + str(intersection(percep_LIWC, list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Space and + 2's: " 
      + str(intersection(space_LIWC, list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Space and - 2's: " 
      + str(intersection(space_LIWC, list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Time and + 2's: " 
      + str(intersection(time_LIWC, list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Time and - 2's: " 
      + str(intersection(time_LIWC, list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Cause and + 2's: " 
      + str(intersection(cause_LIWC, list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Cause and - 2's: " 
      + str(intersection(cause_LIWC, list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Comparatives and + 2's: " 
      + str(intersection(comp_LIWC, list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Comparatives and - 2's: " 
      + str(intersection(comp_LIWC, list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Money and + 2's: " 
      + str(intersection(money_LIWC, list(corrects_2[corrects_2['attr']>0]['vocab'])))+ '\n')
print("Overlap between Money and - 2's: " 
      + str(intersection(money_LIWC, list(corrects_2[corrects_2['attr']<=0]['vocab'])))+ '\n')


Overlap between Cognitive and + 2's: ['nothing', 'nowhere', 'barely', 'didnt', 'supposed', 'never', 'solve', 'neither', 'sort', 'positive', 'lacking', 'somewhere', 'thinks', 'not']

Overlap between Cognitive Person and - 2's: ['becomes', 'perfect', 'outstanding', 'overall', 'always']

Overlap between Perception Person and + 2's: ['burn', 'ear', 'noisy', 'dark', 'loud']

Overlap between Perception and - 2's: ['beautiful']

Overlap between Space and + 2's: ['small', 'nowhere', 'below', 'thin', 'downhill', 'stuck', 'tiny', 'elsewhere', 'somewhere']

Overlap between Space and - 2's: ['vast']

Overlap between Time and + 2's: ['dated', 'never', 'constant', 'old']

Overlap between Time and - 2's: ['always']

Overlap between Cause and + 2's: []

Overlap between Cause and - 2's: []

Overlap between Comparatives and + 2's: ['worse', 'worst', 'dirtiest', 'neither']

Overlap between Comparatives and - 2's: ['best', 'easier']

Overlap between Money and + 2's: ['poorly', 'expenses']

Overlap between

In [110]:
print("Overlap between Sad and + 3's: " 
      + str(intersection(sad_LIWC, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between Sad and - 3's: " 
      + str(intersection(sad_LIWC, list(corrects_3[corrects_3['attr']<=0]['vocab']))))
print("Overlap between Anger and + 3's: " 
      + str(intersection(anger_LIWC, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between Anger and - 3's: " 
      + str(intersection(anger_LIWC, list(corrects_3[corrects_3['attr']<=0]['vocab']))))
print("Overlap between Anxiety and + 3's: " 
      + str(intersection(anxiety_LIWC, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between Anxiety and - 3's: " 
      + str(intersection(anxiety_LIWC, list(corrects_3[corrects_3['attr']<=0]['vocab']))))
print("Overlap between Negemo and + 3's: " 
      + str(intersection(negemo_LIWC, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between Negemo and - 3's: " 
      + str(intersection(negemo_LIWC, list(corrects_3[corrects_3['attr']<=0]['vocab']))))
print("Overlap between Posemo and + 3's: " 
      + str(intersection(posemo_LIWC, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between Posemo and - 3's: " 
      + str(intersection(posemo_LIWC, list(corrects_3[corrects_3['attr']<=0]['vocab']))))
print("Overlap between negat and + 3's: " 
      + str(intersection(negat_LIWC, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between negat and - 3's: " 
      + str(intersection(negat_LIWC, list(corrects_3[corrects_3['attr']<=0]['vocab']))))
print("Overlap between adj and + 3's: " 
      + str(intersection(adj_LIWC, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between adj and - 3's: " 
      + str(intersection(adj_LIWC, list(corrects_3[corrects_3['attr']<=0]['vocab']))))

Overlap between Sad and + 3's: []
Overlap between Sad and - 3's: []
Overlap between Anger and + 3's: ['rude', 'annoying']
Overlap between Anger and - 3's: ['tricky']
Overlap between Anxiety and + 3's: []
Overlap between Anxiety and - 3's: []
Overlap between Negemo and + 3's: ['terrible', 'worst', 'awful', 'poor', 'rude', 'unfriendly', 'bad', 'jaded', 'impersonal', 'fake', 'strange', 'tedious', 'annoying']
Overlap between Negemo and - 3's: ['tricky', 'gross', 'awful']
Overlap between Posemo and + 3's: ['fair', 'okay', 'goodness', 'hopes', 'special', 'ok', 'nice', 'fine', 'decent']
Overlap between Posemo and - 3's: ['helpful', 'thank', 'comfortable', 'fab', 'loved', 'fantastic', 'lovely', 'great', 'nice', 'amazing', 'thanks', 'surprise', 'good', 'free', 'value', 'happy', 'excellent', 'beautiful', 'nicely']
Overlap between negat and + 3's: ['nothing', 'no', 'not']
Overlap between negat and - 3's: ['never']
Overlap between adj and + 3's: ['small', 'worst', 'dirty', 'strange', 'typical', 's

In [57]:
print("Rating 3 vs NRC")
print("Overlap between Sad and + 3's: " 
      + str(intersection(list(sadness_NRC), list(corrects_3[corrects_3['attr']>0]['vocab']))) + '\n')
print("Overlap between Sad and - 3's: " 
      + str(intersection(list(sadness_NRC), list(corrects_3[corrects_3['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Anger and + 3's: " 
      + str(intersection(list(anger_NRC), list(corrects_3[corrects_3['attr']>0]['vocab'])))+ '\n')
print("Overlap between Anger and - 3's: " 
      + str(intersection(list(anger_NRC), list(corrects_3[corrects_3['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Fear and + 3's: " 
      + str(intersection(list(fear_NRC), list(corrects_3[corrects_3['attr']>0]['vocab'])))+ '\n')
print("Overlap between Fear and - 3's: " 
      + str(intersection(list(fear_NRC), list(corrects_3[corrects_3['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Trust and + 3's: " 
      + str(intersection(list(trust_NRC), list(corrects_3[corrects_3['attr']>0]['vocab'])))+ '\n')
print("Overlap between Trust and - 3's: " 
      + str(intersection(list(trust_NRC), list(corrects_3[corrects_3['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Surprise and + 3's: " 
      + str(intersection(list(surprise_NRC), list(corrects_3[corrects_3['attr']>0]['vocab'])))+ '\n')
print("Overlap between Surprise and - 3's: " 
      + str(intersection(list(surprise_NRC), list(corrects_3[corrects_3['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Anticipation and + 3's: " 
      + str(intersection(list(anticipation_NRC), list(corrects_3[corrects_3['attr']>0]['vocab'])))+ '\n')
print("Overlap between Anticipation and - 3's: " 
      + str(intersection(list(anticipation_NRC), list(corrects_3[corrects_3['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Disgust and +3's: " 
      + str(intersection(list(disgust_NRC), list(corrects_3[corrects_3['attr']>0]['vocab'])))+ '\n')
print("Overlap between Disgust and - 3's: " 
      + str(intersection(list(disgust_NRC), list(corrects_3[corrects_3['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Neg and + 3's: " 
      + str(intersection(list(neg_NRC), list(corrects_3[corrects_3['attr']>0]['vocab'])))+ '\n')
print("Overlap between Neg and - 3's: " 
      + str(intersection(list(neg_NRC), list(corrects_3[corrects_3['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Pos and + 3's: " 
      + str(intersection(list(pos_NRC), list(corrects_3[corrects_3['attr']>0]['vocab'])))+ '\n')
print("Overlap between Pos and - 3's: " 
      + str(intersection(list(pos_NRC), list(corrects_3[corrects_3['attr']<=0]['vocab'])))+ '\n')

Rating 3 vs NRC
Overlap between Sad and + 3's: ['terrible', 'inconvenient', 'disappointing', 'awful', 'disappointed', 'depressing', 'unfriendly', 'broken', 'older', 'dark', 'limited', 'illness', 'bad']

Overlap between Sad and - 3's: ['lovely', 'quiet', 'awful']

Overlap between Anger and + 3's: ['stolen', 'terrible', 'inconvenient', 'cracked', 'awful', 'surly', 'misleading', 'noisy', 'unfriendly', 'bad', 'ridiculous', 'broken', 'limited', 'disappointed', 'pirate', 'annoying']

Overlap between Anger and - 3's: ['complaint', 'awful']

Overlap between Fear and + 3's: ['terrible', 'nightmare', 'cracked', 'awful', 'unfriendly', 'broken', 'illness', 'bad']

Overlap between Fear and - 3's: ['surprise', 'snake', 'awful']

Overlap between Trust and + 3's: ['grant', 'goodness', 'fairly', 'expect']

Overlap between Trust and - 3's: ['assured', 'helpful', 'clean', 'lovely', 'recommend', 'good', 'happy', 'excellent']

Overlap between Surprise and + 3's: ['goodness', 'expect', 'guess']

Overlap bet

In [62]:
print("Overlap between First Person and + 3's: " 
      + str(intersection(fp, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between First Person and - 3's: " 
      + str(intersection(fp, list(corrects_3[corrects_3['attr']<=0]['vocab']))))
print("Overlap between Third Person and + 3's: " 
      + str(intersection(tp, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between Third Person and - 3's: " 
      + str(intersection(tp, list(corrects_3[corrects_3['attr']<=0]['vocab']))))
print("Overlap between Narrative Seq and + 3's: " 
      + str(intersection(narr, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between Narrative Seq and - 3's: " 
      + str(intersection(narr, list(corrects_3[corrects_3['attr']<=0]['vocab']))))


Overlap between First Person and + 3's: []
Overlap between First Person and - 3's: []
Overlap between Third Person and + 3's: []
Overlap between Third Person and - 3's: []
Overlap between Narrative Seq and + 3's: []
Overlap between Narrative Seq and - 3's: []


In [111]:
print("Overlap between Cognition and + 3's: " 
      + str(intersection(cogproc_LIWC, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between Cognition and - 3's: " 
      + str(intersection(cogproc_LIWC, list(corrects_3[corrects_3['attr']<=0]['vocab']))))
print("Overlap between Perception and + 3's: " 
      + str(intersection(percep_LIWC, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between Perception and - 3's: " 
      + str(intersection(percep_LIWC, list(corrects_3[corrects_3['attr']<=0]['vocab']))))
print("Overlap between Space and + 3's: " 
      + str(intersection(space_LIWC, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between Space and - 3's: " 
      + str(intersection(space_LIWC, list(corrects_3[corrects_3['attr']<=0]['vocab']))))
print("Overlap between Time and + 3's: " 
      + str(intersection(time_LIWC, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between Time and - 3's: " 
      + str(intersection(time_LIWC, list(corrects_3[corrects_3['attr']<=0]['vocab']))))
print("Overlap between Cause and + 3's: " 
      + str(intersection(cause_LIWC, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between Cause and - 3's: " 
      + str(intersection(cause_LIWC, list(corrects_3[corrects_3['attr']<=0]['vocab']))))
print("Overlap between Comparatives and + 3's: " 
      + str(intersection(comp_LIWC, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between Comparatives and - 3's: " 
      + str(intersection(comp_LIWC, list(corrects_3[corrects_3['attr']<=0]['vocab']))))
print("Overlap between Money and + 3's: " 
      + str(intersection(money_LIWC, list(corrects_3[corrects_3['attr']>0]['vocab']))))
print("Overlap between Money and - 3's: " 
      + str(intersection(money_LIWC, list(corrects_3[corrects_3['attr']<=0]['vocab']))))



Overlap between Cognition and + 3's: ['nothing', 'positives', 'opinion', 'somewhere', 'lack', 'think', 'hopes', 'depends', 'needs', 'lacking', 'however', 'though', 'barely', 'not']
Overlap between Cognition and - 3's: ['affect', 'tricky', 'never', 'outstanding']
Overlap between Perception and + 3's: ['ear', 'noisy', 'dark', 'noise', 'loud']
Overlap between Perception and - 3's: ['quiet', 'beautiful']
Overlap between Space and + 3's: ['mixed', 'small', 'thin', 'tiny', 'somewhere']
Overlap between Space and - 3's: ['right', 'downhill', 'town', 'stay', 'back']
Overlap between Time and + 3's: ['slow', 'firstly', 'busy', 'dated', 'forever', 'older', 'starting', 'old']
Overlap between Time and - 3's: ['never', 'immediately', 'back']
Overlap between Cause and + 3's: []
Overlap between Cause and - 3's: []
Overlap between Comparatives and + 3's: ['less', 'worst', 'older']
Overlap between Comparatives and - 3's: []
Overlap between Money and + 3's: ['expensive']
Overlap between Money and - 3's: [

In [113]:
print("Overlap between Sad and + 4's: " 
      + str(intersection(sad_LIWC, list(corrects_4[corrects_4['attr']>0]['vocab']))))
print("Overlap between Sad and - 4's: " 
      + str(intersection(sad_LIWC, list(corrects_4[corrects_4['attr']<=0]['vocab']))))
print("Overlap between Anger and + 4's: " 
      + str(intersection(anger_LIWC, list(corrects_4[corrects_4['attr']>0]['vocab']))))
print("Overlap between Anger and - 4's: " 
      + str(intersection(anger_LIWC, list(corrects_4[corrects_4['attr']<=0]['vocab']))))
print("Overlap between Anxiety and + 4's: " 
      + str(intersection(anxiety_LIWC, list(corrects_4[corrects_4['attr']>0]['vocab']))))
print("Overlap between Anxiety and - 4's: " 
      + str(intersection(anxiety_LIWC, list(corrects_4[corrects_4['attr']<=0]['vocab']))))
print("Overlap between Negemo and + 4's: " 
      + str(intersection(negemo_LIWC, list(corrects_4[corrects_4['attr']>0]['vocab']))))
print("Overlap between Negemo and - 4's: " 
      + str(intersection(negemo_LIWC, list(corrects_4[corrects_4['attr']<=0]['vocab']))))
print("Overlap between Posemo and + 4's: " 
      + str(intersection(posemo_LIWC, list(corrects_4[corrects_4['attr']>0]['vocab']))))
print("Overlap between Posemo and - 4's: " 
      + str(intersection(posemo_LIWC, list(corrects_4[corrects_4['attr']<=0]['vocab']))))
print("Overlap between negat and + 4's: " 
      + str(intersection(negat_LIWC, list(corrects_4[corrects_4['attr']>0]['vocab']))))
print("Overlap between negat and - 4's: " 
      + str(intersection(negat_LIWC, list(corrects_4[corrects_4['attr']<=0]['vocab']))))
print("Overlap between adj and + 4's: " 
      + str(intersection(adj_LIWC, list(corrects_4[corrects_4['attr']>0]['vocab']))))
print("Overlap between adj and - 4's: " 
      + str(intersection(adj_LIWC, list(corrects_4[corrects_4['attr']<=0]['vocab']))))

Overlap between Sad and + 4's: ['lame']
Overlap between Sad and - 4's: ['low']
Overlap between Anger and + 4's: ['rude', 'annoying']
Overlap between Anger and - 4's: ['critical']
Overlap between Anxiety and + 4's: ['horrible', 'confusing']
Overlap between Anxiety and - 4's: []
Overlap between Negemo and + 4's: ['sadly', 'confusing', 'lame', 'boring', 'horrible', 'rude', 'difficult', 'unfriendly', 'bad', 'badly', 'hell', 'impersonal', 'poorly', 'annoying']
Overlap between Negemo and - 4's: ['serious', 'lazy', 'poor', 'unpleasant', 'critical', 'low', 'mess', 'bad']
Overlap between Posemo and + 4's: ['perfect', 'okay', 'liked', 'liking', 'giving', 'fantastic', 'loved', 'pretty', 'great', 'nice', 'beautiful', 'excellent', 'fine', 'fun', 'good', 'value', 'comfortable', 'lovely', 'decent', 'nicely']
Overlap between Posemo and - 4's: ['comfort', 'won', 'fav', 'super', 'love', 'glad', 'luck', 'beautiful', 'brilliant', 'importantly', 'cares', 'loved', 'excited', 'ok', 'pleasing', 'excellent', '

In [58]:
print("Rating 4 vs NRC")
print("Overlap between Sad and + 4's: " 
      + str(intersection(list(sadness_NRC), list(corrects_4[corrects_4['attr']>0]['vocab']))) + '\n')
print("Overlap between Sad and - 4's: " 
      + str(intersection(list(sadness_NRC), list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Anger and + 4's: " 
      + str(intersection(list(anger_NRC), list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Anger and - 4's: " 
      + str(intersection(list(anger_NRC), list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Fear and + 4's: " 
      + str(intersection(list(fear_NRC), list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Fear and - 4's: " 
      + str(intersection(list(fear_NRC), list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Trust and + 4's: " 
      + str(intersection(list(trust_NRC), list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Trust and - 4's: " 
      + str(intersection(list(trust_NRC), list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Surprise and + 4's: " 
      + str(intersection(list(surprise_NRC), list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Surprise and - 4's: " 
      + str(intersection(list(surprise_NRC), list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Anticipation and + 4's: " 
      + str(intersection(list(anticipation_NRC), list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Anticipation and - 4's: " 
      + str(intersection(list(anticipation_NRC), list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Disgust and +4's: " 
      + str(intersection(list(disgust_NRC), list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Disgust and - 4's: " 
      + str(intersection(list(disgust_NRC), list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Neg and + 4's: " 
      + str(intersection(list(neg_NRC), list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Neg and - 4's: " 
      + str(intersection(list(neg_NRC), list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Pos and + 4's: " 
      + str(intersection(list(pos_NRC), list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Pos and - 4's: " 
      + str(intersection(list(pos_NRC), list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')

Rating 4 vs NRC
Overlap between Sad and + 4's: ['negative', 'sadly', 'overpriced', 'overwhelmed', 'hell', 'inconvenient', 'disturbed', 'complain', 'unfriendly', 'badly', 'quiet', 'terminal', 'disappointment', 'criticism', 'dark', 'limited', 'problem', 'fault', 'disappointing', 'shame', 'overload', 'older', 'lovely', 'bad']

Overlap between Sad and - 4's: ['doubt', 'nasty', 'disappointing', 'regret', 'disappointed', 'worn', 'unpleasant', 'shack', 'dark', 'quiet', 'disappoint', 'homeless', 'leave', 'bad']

Overlap between Anger and + 4's: ['blast', 'inconvenient', 'disturbed', 'complain', 'horrible', 'criticism', 'noisy', 'unfriendly', 'overpriced', 'complaint', 'annoyance', 'bad', 'hell', 'limited', 'annoying']

Overlap between Anger and - 4's: ['nasty', 'noisy', 'disappoint', 'homeless', 'disappointed', 'bad']

Overlap between Fear and + 4's: ['cutter', 'blast', 'shame', 'horrible', 'difficult', 'unfriendly', 'flood', 'beware', 'hell', 'terminal', 'problem', 'bad']

Overlap between Fea

In [64]:
print("Overlap between First Person and + 4's: " 
      + str(intersection(fp, list(corrects_4[corrects_4['attr']>0]['vocab']))) + '\n')
print("Overlap between First Person and - 4's: " 
      + str(intersection(fp, list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Third Person and + 4's: " 
      + str(intersection(tp, list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Third Person and - 4's: " 
      + str(intersection(tp, list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Narrative Seq and + 4's: " 
      + str(intersection(narr, list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Narrative Seq and - 4's: " 
      + str(intersection(narr, list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')


Overlap between First Person and + 4's: []

Overlap between First Person and - 4's: []

Overlap between Third Person and + 4's: []

Overlap between Third Person and - 4's: []

Overlap between Narrative Seq and + 4's: []

Overlap between Narrative Seq and - 4's: ['say']



In [117]:
print("Overlap between Cognition and + 4's: " 
      + str(intersection(cogproc_LIWC, list(corrects_4[corrects_4['attr']>0]['vocab']))) + '\n')
print("Overlap between Cognition and - 4's: " 
      + str(intersection(cogproc_LIWC, list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Perception and + 4's: " 
      + str(intersection(percep_LIWC, list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Perception and - 4's: " 
      + str(intersection(percep_LIWC, list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Space and + 4's: " 
      + str(intersection(space_LIWC, list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Space and - 4's: " 
      + str(intersection(space_LIWC, list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Time and + 4's: " 
      + str(intersection(time_LIWC, list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Time and - 4's: " 
      + str(intersection(time_LIWC, list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Cause and + 4's: " 
      + str(intersection(cause_LIWC, list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Cause and - 4's: " 
      + str(intersection(cause_LIWC, list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Comparatives and + 4's: " 
      + str(intersection(comp_LIWC, list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Comparatives and - 4's: " 
      + str(intersection(comp_LIWC, list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Money and + 4's: " 
      + str(intersection(money_LIWC, list(corrects_4[corrects_4['attr']>0]['vocab'])))+ '\n')
print("Overlap between Money and - 4's: " 
      + str(intersection(money_LIWC, list(corrects_4[corrects_4['attr']<=0]['vocab'])))+ '\n')


Overlap between Cognition and + 4's: ['nowhere', 'odd', 'lack', 'neither', 'overall', 'apart', 'lacking', 'however', 'particularly', 'apparently', 'suppose', 'some', 'sometimes', 'although', 'though', 'would', 'lacks', 'pretty', 'almost', 'should', 'perfect', 'otherwise', 'seemed', 'not']

Overlap between Cognition and - 4's: ['how', 'definitely', 'perfect', 'anything', 'positives', 'outstanding', 'ever', 'never', 'always', 'needs', 'nothing', 'not', 'somewhere', 'anywhere']

Overlap between Perception and + 4's: ['seeing', 'hear', 'speak', 'ear', 'noisy', 'beautiful', 'louder', 'quiet', 'dark', 'loudest', 'loud']

Overlap between Perception and - 4's: ['say', 'noisy', 'quiet', 'dark', 'beautiful', 'felt']

Overlap between Space and + 4's: ['small', 'nowhere', 'large', 'flat', 'biggest', 'thin', 'side', 'massive', 'near', 'tiny', 'apart', 'beside', 'sits', 'downtown', 'little']

Overlap between Space and - 4's: ['small', 'longer', 'low', 'stuck', 'beyond', 'staying', 'somewhere', 'anyw

In [116]:
print("Overlap between Sad and + 5's: " 
      + str(intersection(sad_LIWC, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between Sad and - 5's: " 
      + str(intersection(sad_LIWC, list(corrects_5[corrects_5['attr']<=0]['vocab']))))
print("Overlap between Anger and + 5's: " 
      + str(intersection(anger_LIWC, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between Anger and - 5's: " 
      + str(intersection(anger_LIWC, list(corrects_5[corrects_5['attr']<=0]['vocab']))))
print("Overlap between Anxiety and + 5's: " 
      + str(intersection(anxiety_LIWC, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between Anxiety and - 5's: " 
      + str(intersection(anxiety_LIWC, list(corrects_5[corrects_5['attr']<=0]['vocab']))))
print("Overlap between Negemo and + 5's: " 
      + str(intersection(negemo_LIWC, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between Negemo and - 's: " 
      + str(intersection(negemo_LIWC, list(corrects_5[corrects_5['attr']<=0]['vocab']))))
print("Overlap between Posemo and + 5's: " 
      + str(intersection(posemo_LIWC, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between Posemo and - 5's: " 
      + str(intersection(posemo_LIWC, list(corrects_5[corrects_5['attr']<=0]['vocab']))))
print("Overlap between negat and + 5's: " 
      + str(intersection(negat_LIWC, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between negat and - 5's: " 
      + str(intersection(negat_LIWC, list(corrects_5[corrects_5['attr']<=0]['vocab']))))
print("Overlap between adj and + 5's: " 
      + str(intersection(adj_LIWC, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between adj and - 5's: " 
      + str(intersection(adj_LIWC, list(corrects_5[corrects_5['attr']<=0]['vocab']))))

Overlap between Sad and + 5's: []
Overlap between Sad and - 5's: ['broke', 'gloomy']
Overlap between Anger and + 5's: []
Overlap between Anger and - 5's: ['annoying']
Overlap between Anxiety and + 5's: []
Overlap between Anxiety and - 5's: []
Overlap between Negemo and + 5's: ['emotional', 'worry', 'worried']
Overlap between Negemo and - 's: ['worse', 'worthless', 'worst', 'sadly', 'awful', 'poor', 'shy', 'gloomy', 'difficult', 'lowered', 'bad', 'missing', 'broke', 'sad', 'strange', 'annoying']
Overlap between Posemo and + 5's: ['lovely', 'fabulous', 'excellence', 'happier', 'determined', 'fav', 'super', 'love', 'care', 'beautiful', 'gorgeously', 'brilliant', 'importantly', 'strongly', 'perfection', 'loved', 'finest', 'superior', 'terrific', 'gorgeous', 'sweet', 'nicest', 'splendor', 'excellent', 'beauty', 'caring', 'valued', 'thank', 'eager', 'awesome', 'fun', 'thanks', 'happy', 'favorite', 'clever', 'definitely', 'perfect', 'truly', 'splendid', 'helpful', 'best', 'wins', 'fab', 'true

In [59]:
print("Rating 5 vs NRC")
print("Overlap between Sad and + 5's: " 
      + str(intersection(list(sadness_NRC), list(corrects_5[corrects_5['attr']>0]['vocab']))) + '\n')
print("Overlap between Sad and - 5's: " 
      + str(intersection(list(sadness_NRC), list(corrects_5[corrects_5['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Anger and + 5's: " 
      + str(intersection(list(anger_NRC), list(corrects_5[corrects_5['attr']>0]['vocab'])))+ '\n')
print("Overlap between Anger and - 5's: " 
      + str(intersection(list(anger_NRC), list(corrects_5[corrects_5['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Fear and + 5's: " 
      + str(intersection(list(fear_NRC), list(corrects_5[corrects_5['attr']>0]['vocab'])))+ '\n')
print("Overlap between Fear and - 5's: " 
      + str(intersection(list(fear_NRC), list(corrects_5[corrects_5['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Trust and + 5's: " 
      + str(intersection(list(trust_NRC), list(corrects_5[corrects_5['attr']>0]['vocab'])))+ '\n')
print("Overlap between Trust and - 5's: " 
      + str(intersection(list(trust_NRC), list(corrects_5[corrects_5['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Surprise and + 5's: " 
      + str(intersection(list(surprise_NRC), list(corrects_5[corrects_5['attr']>0]['vocab'])))+ '\n')
print("Overlap between Surprise and - 5's: " 
      + str(intersection(list(surprise_NRC), list(corrects_5[corrects_5['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Anticipation and + 5's: " 
      + str(intersection(list(anticipation_NRC), list(corrects_5[corrects_5['attr']>0]['vocab'])))+ '\n')
print("Overlap between Anticipation and - 5's: " 
      + str(intersection(list(anticipation_NRC), list(corrects_5[corrects_5['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Disgust and +5's: " 
      + str(intersection(list(disgust_NRC), list(corrects_5[corrects_5['attr']>0]['vocab'])))+ '\n')
print("Overlap between Disgust and - 5's: " 
      + str(intersection(list(disgust_NRC), list(corrects_5[corrects_5['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Neg and + 5's: " 
      + str(intersection(list(neg_NRC), list(corrects_5[corrects_5['attr']>0]['vocab'])))+ '\n')
print("Overlap between Neg and - 5's: " 
      + str(intersection(list(neg_NRC), list(corrects_5[corrects_5['attr']<=0]['vocab'])))+ '\n')
print("Overlap between Pos and + 5's: " 
      + str(intersection(list(pos_NRC), list(corrects_5[corrects_5['attr']>0]['vocab'])))+ '\n')
print("Overlap between Pos and - 5's: " 
      + str(intersection(list(pos_NRC), list(corrects_5[corrects_5['attr']<=0]['vocab'])))+ '\n')

Rating 5 vs NRC
Overlap between Sad and + 5's: ['die', 'regret', 'fault', 'worry', 'leave', 'quiet', 'worried', 'disappoint', 'terrific', 'lovely', 'bomb']

Overlap between Sad and - 5's: ['worse', 'sadly', 'execution', 'depressing', 'overpriced', 'impossible', 'tough', 'missing', 'broken', 'worthless', 'neglected', 'disappointment', 'gloomy', 'hanging', 'dark', 'limited', 'disaster', 'death', 'disappointing', 'lax', 'awful', 'shame', 'dreary', 'older', 'broke', 'disappointed', 'bad']

Overlap between Anger and + 5's: ['blast', 'rave', 'commotion', 'raving', 'disappoint', 'bomb']

Overlap between Anger and - 5's: ['death', 'worthless', 'neglected', 'awful', 'surly', 'smell', 'execution', 'noisy', 'overpriced', 'bad', 'ridiculous', 'broken', 'hanging', 'limited', 'disappointed', 'disaster', 'annoying']

Overlap between Fear and + 5's: ['blast', 'die', 'marry', 'worry', 'graduation', 'hesitation', 'raving', 'khan', 'bomb']

Overlap between Fear and - 5's: ['warning', 'worse', 'death', 'a

In [65]:
print("Overlap between First Person and + 5's: " 
      + str(intersection(fp, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between First Person and - 5's: " 
      + str(intersection(fp, list(corrects_5[corrects_5['attr']<=0]['vocab']))))
print("Overlap between Third Person and + 5's: " 
      + str(intersection(tp, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between Third Person and - 5's: " 
      + str(intersection(tp, list(corrects_5[corrects_5['attr']<=0]['vocab']))))
print("Overlap between Narrative Seq and + 5's: " 
      + str(intersection(narr, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between Narrative Seq and - 5's: " 
      + str(intersection(narr, list(corrects_5[corrects_5['attr']<=0]['vocab']))))

Overlap between First Person and + 5's: []
Overlap between First Person and - 5's: []
Overlap between Third Person and + 5's: []
Overlap between Third Person and - 5's: []
Overlap between Narrative Seq and + 5's: ['initially']
Overlap between Narrative Seq and - 5's: ['telling']


In [115]:
print("Overlap between Cognition and + 5's: " 
      + str(intersection(cogproc_LIWC, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between Cognition and - 5's: " 
      + str(intersection(cogproc_LIWC, list(corrects_5[corrects_5['attr']<=0]['vocab']))))
print("Overlap between Perception and + 5's: " 
      + str(intersection(percep_LIWC, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between Perception and - 5's: " 
      + str(intersection(percep_LIWC, list(corrects_5[corrects_5['attr']<=0]['vocab']))))
print("Overlap between Space and + 5's: " 
      + str(intersection(time_LIWC, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between Space and - 5's: " 
      + str(intersection(time_LIWC, list(corrects_5[corrects_5['attr']<=0]['vocab']))))
print("Overlap between Time and + 5's: " 
      + str(intersection(time_LIWC, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between Time and - 5's: " 
      + str(intersection(time_LIWC, list(corrects_5[corrects_5['attr']<=0]['vocab']))))
print("Overlap between Cause and + 5's: " 
      + str(intersection(cause_LIWC, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between Cause and - 5's: " 
      + str(intersection(cause_LIWC, list(corrects_5[corrects_5['attr']<=0]['vocab']))))
print("Overlap between Comparatives and + 5's: " 
      + str(intersection(comp_LIWC, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between Comparatives and - 5's: " 
      + str(intersection(comp_LIWC, list(corrects_5[corrects_5['attr']<=0]['vocab']))))
print("Overlap between Money and + 5's: " 
      + str(intersection(money_LIWC, list(corrects_5[corrects_5['attr']>0]['vocab']))))
print("Overlap between Money and - 5's: " 
      + str(intersection(money_LIWC, list(corrects_5[corrects_5['attr']<=0]['vocab']))))



Overlap between Cognition and + 5's: ['definitely', 'perfect', 'bosses', 'truly', 'everytime', 'outstanding', 'absolutely', 'true', 'anytime', 'secrets', 'determined', 'anywhere', 'always']
Overlap between Cognition and - 5's: ['nowhere', 'supposedly', 'lack', 'overall', 'impossible', 'lacking', 'however', 'barely', 'though', 'lacked', 'would', 'admit', 'nevertheless', 'pretty', 'positive', 'needs', 'almost', 'somewhere', 'should', 'otherwise', 'seemed', 'nothing', 'not']
Overlap between Perception and + 5's: ['coolest', 'beauty', 'speaks', 'violet', 'quiet', 'warmly', 'sweet', 'beautiful', 'hands']
Overlap between Perception and - 5's: ['shadow', 'gloomy', 'noisy', 'sour', 'tight', 'dark', 'noise', 'loud']
Overlap between Space and + 5's: ['soon', 'quickly', 'whenever', 'everytime', 'forward', 'anytime', 'fast', 'modern', 'finish', 'immediately', 'always', 'already', 'continues', 'quickest']
Overlap between Space and - 5's: ['slow', 'busy', 'dated', 'forever', 'present', 'slower', 'co